In [4]:
import tensorflow as tf
import PIL as pil
from PIL import ImageOps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers,Sequential, callbacks, losses
import os
import random


data preprocessing (remove the suroundding noises and emphasize the crack)

In [ ]:
def img_to_array(img_path, augmented=False) :
    width = 128
    height = 128
    img = pil.Image.open(img_path)
    img = (np.array(img,np.int32) + 120) / 255
    img = tf.image.adjust_contrast(img,3)
    img = tf.image.resize(img,(width,height))
    arr = []
    arr.append(img)
    if(augmented) : 
        arr.append(img)
        for i in range(3) :
            img = tf.image.rot90(img)
            arr.append(img)
    return arr


function that prepares the data batch

In [ ]:
def batch_preparation() :
    cracked_path = "../../Decks/Cracked/"
    not_cracked_path = "../../Decks/Non-Cracked/"

    cracked = os.listdir(cracked_path) 
    cracked_len = len(cracked) - 1
    not_cracked = os.listdir(not_cracked_path)
    not_cracked_len = len(not_cracked) - 1

    images_dict = {}

    cracked_num = random.randint(0,2000)
    not_cracked_num = 2000 - cracked_num

    #load in the files in cracked deck file
    for i in range(cracked_num) :
        file = cracked[random.randint(0,cracked_len)];
        img = img_to_array(cracked_path + file, augmented=False)
        for i in img:
            images_dict.update({i.ref():1})

    for i in range(not_cracked_num) :
        file = not_cracked[random.randint(0,not_cracked_len)];
        img = img_to_array(not_cracked_path + file, augmented=False)
        for i in img:
            images_dict.update({i.ref():0})
    
    return images_dict


prepare the data for training

In [ ]:
imgs = batch_preparation()

cols = ["isCracked"]
train_data = pd.DataFrame.from_dict(imgs,orient='index', columns=cols)

sampled = train_data.sample(n=2000,random_state = 1,replace = False)

derefered_images = []
train_images = np.array(sampled.index.values.tolist()) #<- list of list

for tensor in train_images:
    derefered_images.append(np.array(tensor.deref()))

train_images = np.array(derefered_images)
train_labels = np.array(sampled['isCracked'].tolist())


build the model

In [7]:
model = Sequential([
    layers.Conv2D(128,(3,3),activation="relu",input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(256,(5,5),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(256,(5,5),activation="relu"),
    layers.Flatten(),
    layers.Dense(32,activation="relu"),
    layers.Dense(1,activation="sigmoid")
])


compile the model

In [ ]:
model.compile(
    optimizer="adam", 
    #loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    #loss='binary_crossentropy',
    loss=losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)


model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoints/modeld.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False
)


train the model

In [ ]:
train_history = model.fit(train_images, train_labels, epochs=30, callbacks=[model_checkpoint_callback],batch_size = 250)


save the model

In [ ]:
model.save("modeld.h5")



function for prediction

In [ ]:
label_names = ["Non-cracked","Cracked"]
def predict_custom_img(img_path : str) : #use to predict custom images
    img = img_to_array(img_path)
    img = np.array(img).reshape((1,128,128,3))
    model.load_weights("modeld.h5")
    #print(model.summary())
    model.save("finalModel.h5")
    prediction = model.predict(img)
    print(prediction)
    # prediction_index = np.argmax(prediction[0])
    # print("Prediction : {}".format(label_names[prediction_index]))
    plt.imshow(img[0])
    plt.show()


function for accuracy benchmark

In [ ]:
def testingAccuracy():
    crackedAccuList = []
    notCrackedAccuList = []
    cracked_path = "../../Decks/Cracked/"
    not_cracked_path = "../../Decks/Non-Cracked/"
    cracked = os.listdir(cracked_path) 
    not_cracked = os.listdir(not_cracked_path)
    model = models.load_model("finalModel.h5")
    for i in range(0,len(cracked),20):
        testImage = img_to_array(cracked_path + cracked[i])
        testImage = np.array(testImage).reshape(1,128,128,3)
        prediction = model.predict(testImage)
        crackedAccuList.append(prediction[0][0])
        print("Cracked Prediction {}, accuracy: {}".format(i,prediction[0]))
    for i in range(0,len(not_cracked),150):
        testImage = img_to_array(not_cracked_path + not_cracked[i])
        testImage = np.array(testImage).reshape(1,128,128,3)
        prediction = model.predict(testImage)
        notCrackedAccuList.append(prediction[0][0])
        print("Not Cracked Prediction {}, accuracy: {}".format(i,prediction[0]))
    crackedAccu = np.array(crackedAccuList).mean()
    notCrackedAccu = np.array(notCrackedAccuList).mean()
    print("Average prediction for cracked: {}".format(crackedAccu))
    print("Average prediction for non-cracked: {}".format(notCrackedAccu))

testingAccuracy()
